In [1]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [2]:
import shutil
shutil.copytree('/content/drive/MyDrive/Data','/content/Data')

'/content/Data'

In [3]:
import pandas as pd
import re

In [4]:
import numpy as np

In [5]:
with open('Data/text-and-id.txt','r') as file:
    lines=file.readlines()

data=[]
for line in lines:
    line_parts=line.split()
    id=int(line_parts[0])
    sentence=' '.join(line_parts[1:])
    data.append((id,sentence))


In [6]:
df=pd.DataFrame(data,columns=["id","sentence"])
df['id']=df['id'].astype(int)
df=df.sort_values('id',ascending=True)

In [7]:
df.head()

,id,sentence
957,1,Besides these forests naturally developed fore...
1607,2,Your mother cleans the rice by separating husk...
1083,3,All free gifts of nature and all things which ...
1412,4,Humanism - the concept that man is the measure...
1344,5,Various designs in clothing are found dependin...


In [8]:
with open('Data/pairs-label-training (1).txt','r') as file:
    lines=file.readlines()

data=[]
for line in lines:
    line_parts=line.split(', ')
    id1=(line_parts[0])
    id2=(line_parts[1])
    line_parts=line.split()
    label=line_parts[2]
    data.append((id1,id2,label))

df1=pd.DataFrame(data,columns=["id1","id2","label"])


In [9]:
df1

,id1,id2,label
0,98,651,0
1,696,1870,1
2,1073,1620,0
3,167,633,0
4,4,640,0
...,...,...,...
155946,546,1512,1
155947,720,1902,1
155948,1112,1591,0
155949,396,1472,1


In [10]:
df1['id1']=df1['id1'].astype(int)
df1['id2']=df1['id2'].astype(int)
df1['label']=df1['label'].astype(int)
df1.head()

,id1,id2,label
0,98,651,0
1,696,1870,1
2,1073,1620,0
3,167,633,0
4,4,640,0


In [11]:
data=[]
for index,row in df1.iterrows():
    idx1=row['id1']
    idx2=row['id2']
    sen1=df[df['id']==idx1]['sentence'].values
    sen2=df[df['id']==idx2]['sentence'].values
    label=row['label']
    data.append((sen1,sen2,label))

In [12]:
gender_df=pd.DataFrame(data,columns=['sentence1','sentence2','label'])

In [39]:
df=pd.DataFrame(data,columns=['sentence1','sentence2','label'])

In [13]:
gender_df.head()

,sentence1,sentence2,label
0,[From the very ancient periods men used to app...,[Men and women of all ages participate in this...,0
1,[So he used sharp stones and bones to loosen t...,[Can you imagine a world in which we cannot ru...,1
2,"[In a closed electric circuit, the electric cu...",[Here is a diagram to show what is really happ...,0
3,[She will not be able to understand all these ...,[He must possess all the qualifications necess...,0
4,[Humanism - the concept that man is the measur...,"[Again , he spends his earned money to buy ess...",0


In [14]:
gender_df['sentence1'] = gender_df['sentence1'].apply(lambda x:x[0])
gender_df['sentence2'] = gender_df['sentence2'].apply(lambda x:x[0])


In [15]:
gender_df

,sentence1,sentence2,label
0,From the very ancient periods men used to appl...,Men and women of all ages participate in this ...,0
1,So he used sharp stones and bones to loosen th...,Can you imagine a world in which we cannot run...,1
2,"In a closed electric circuit, the electric cur...",Here is a diagram to show what is really happe...,0
3,She will not be able to understand all these a...,He must possess all the qualifications necessa...,0
4,Humanism - the concept that man is the measure...,"Again , he spends his earned money to buy esse...",0
...,...,...,...
155946,She has to take active steps in implementing t...,The people of India decide their political for...,1
155947,As a result there is increased coastal erosion...,We will keep the school neat and clean .,1
155948,You will see that when water inside and outsid...,Dwarkanath Tagore believed that India would de...,0
155949,Try to observe a blacksmith at work. He beats ...,"Due to this , fountain water is called mineral...",1


In [ ]:
!pip install contractions

In [ ]:
import contractions
import nltk
nltk.download('all')    

In [18]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [19]:
def clean_contraction_mapping(text):
  expanded_words = []   
  for word in text.split():
  # using contractions.fix to expand the shortened words
    expanded_words.append(contractions.fix(word))
  expanded_text = ' '.join(expanded_words)
  return expanded_text  

In [20]:
bias=["he","she","him","her","his","mother","father","sister","brother","male","female","men","women","man","woman","boys","girls","himself","herself"]

In [21]:
stopwords=list(set(stopwords.words('english'))-set(bias))

In [22]:
def clean(text):
  text=clean_contraction_mapping(text)
  text=re.sub('[^a-zA-Z]'," ",text)
  text=text.lower()
  text=text.split()
  text=[lem.lemmatize(word) for word in text if not word in stopwords]
  return " ".join(text)

In [23]:
lem=WordNetLemmatizer()
gender_df['sentence1']=np.vectorize(clean)(gender_df['sentence1'])
gender_df['sentence2']=np.vectorize(clean)(gender_df['sentence2'])

In [24]:
import spacy
from spacy import displacy
ner=spacy.load('en_core_web_sm')

In [25]:
def apply_ner(text):
  text1=ner(text)
  arr=[]
  for word in text1.ents:
    # print(word.text,":",word.label_)
    arr.append(word.label_)
  return arr

In [26]:
text="She will not be able to understand all these aspects in a day ."
apply_ner(gender_df.iloc[3]['sentence1'])

[]

In [27]:
# gender_df['ner1']=gender_df['sentence1'].apply(apply_ner)
# gender_df['ner2']=gender_df['sentence2'].apply(apply_ner)

In [28]:
gender_df

,sentence1,sentence2,label
0,ancient period men used apply ointment face sp...,men woman age participate celebration,0
1,he used sharp stone bone loosen soil learnt pr...,imagine world cannot run motor train,1
2,closed electric circuit electric current pass ...,diagram show really happening compound reactio...,0
3,she able understand aspect day,he must posse qualification necessary become m...,0
4,humanism concept man measure thing,he spends his earned money buy essential good ...,0
...,...,...,...
155946,she take active step implementing activity acc...,people india decide political fortune election,1
155947,result increased coastal erosion activity man ...,keep school neat clean,1
155948,see water inside outside cylinder come level d...,dwarkanath tagore believed india would develop...,0
155949,try observe blacksmith work he beat hot iron p...,due fountain water called mineral water,1


In [29]:
def check(text):
  flag=0
  text1=ner(text)
  arr=[]
  for word in text1.ents:
    # print(word.text,":",word.label_)
    arr.append(word.label_)
  if 'PERSON' in arr:
    flag=1
  for word in text.split():
    if word in bias:
      flag=1
  return flag

In [30]:
gender_df['bias1']=np.vectorize(check)(gender_df['sentence1'])
gender_df['bias2']=np.vectorize(check)(gender_df['sentence2'])

In [31]:
gender_df['bias']=gender_df['bias1']^gender_df['bias2']

In [32]:
gender_df

,sentence1,sentence2,label,bias1,bias2,bias
0,ancient period men used apply ointment face sp...,men woman age participate celebration,0,1,1,0
1,he used sharp stone bone loosen soil learnt pr...,imagine world cannot run motor train,1,1,0,1
2,closed electric circuit electric current pass ...,diagram show really happening compound reactio...,0,0,0,0
3,she able understand aspect day,he must posse qualification necessary become m...,0,1,1,0
4,humanism concept man measure thing,he spends his earned money buy essential good ...,0,1,1,0
...,...,...,...,...,...,...
155946,she take active step implementing activity acc...,people india decide political fortune election,1,1,0,1
155947,result increased coastal erosion activity man ...,keep school neat clean,1,1,0,1
155948,see water inside outside cylinder come level d...,dwarkanath tagore believed india would develop...,0,0,1,1
155949,try observe blacksmith work he beat hot iron p...,due fountain water called mineral water,1,1,0,1


In [33]:
len(gender_df[gender_df['bias'].astype('int64')==gender_df['label']])/len(gender_df)

0.8543324505774249

In [36]:
gender_df[gender_df['label']!=gender_df['bias']]

,sentence1,sentence2,label,bias1,bias2,bias
16,time need modesty scarf loose clothes comforta...,due fountain water called mineral water,1,0,0,0
23,carrying process production ahead man accepted...,think growth spurt took place hint think back ...,0,1,0,1
25,well ahead festival people clean house through...,scouting help make boy girl patriotic good ide...,0,1,0,1
26,school going girl morning school take breakfas...,colonisation vietnam french brought people cou...,1,0,0,0
36,important separate religion state,seed balsam dispersed manner may also noticed ...,0,0,1,1
...,...,...,...,...,...,...
155938,stereotype believe people belonging particular...,process knitting sweater six year old boy desc...,0,1,0,1
155942,characteristic sound help identify friend his ...,ii dwarf broiler parent commercial chick produ...,1,1,1,0
155944,scouting help make boy girl patriotic good ide...,experiment take three sample namely iron piece...,1,0,0,0
155945,man slave habit,v km h,1,1,1,0


In [46]:
gender_df.iloc[25]['sentence1'],gender_df.iloc[25]['sentence2']

('well ahead festival people clean house throughout pongal rangoli day woman decorate house colourful',
 'scouting help make boy girl patriotic good ideal citizen')

In [47]:
df.iloc[25]['sentence1'],df.iloc[25]['sentence2']

(array(['Well ahead of the festival , people clean their houses and throughout Pongal Rangoli days , women decorate their houses with colourful .'],
       dtype=object),
 array(['Scouting helps make the boys and girls patriotic , good and ideal citizens .'],
       dtype=object))

In [48]:
apply_ner(str(df.iloc[25]['sentence2']))

[]